### import 

In [1]:
import pandas as pd
import numpy as np
import math
import random
import os

In [50]:
from sklearn.preprocessing import MinMaxScaler

In [51]:
position_scaler = MinMaxScaler().fit([[0], [720]])

### Data Loading

In [6]:
data = pd.read_csv("out.csv").drop("Unnamed: 0", axis=1)

In [7]:
data = data.T

In [4]:
columns = ["health", "health", "health", "health", "health", "health", 
           "ammo", "ammo", "ammo", 
           "alive", "current_x", "current_y", "end_x", "end_y",
           "alive", "current_x", "current_y", "end_x", "end_y",
           "alive", "current_x", "current_y", "end_x", "end_y",
           "alive", "current_x", "current_y", "end_x", "end_y",
           "alive", "current_x", "current_y", "end_x", "end_y",
           "alive", "current_x", "current_y", "end_x", "end_y",
           "alive", "current_x", "current_y", "end_x", "end_y",
           "alive", "current_x", "current_y", "end_x", "end_y",
           "alive", "current_x", "current_y", "end_x", "end_y",
           "alive", "current_x", "current_y", "end_x", "end_y",
          ]

In [5]:
data.columns = columns

In [6]:
def sigmoid(x):
    return 1 / (1 + math.exp(-x))

### Model Building 

In [10]:
class Model():
    def __init__(self, weights=None):
        """When class is created, it is either created with previos weights, or randomly initiated

        -Input size is 59 (of data points)
        -Output size is 3 (shoot, x and y)
        -Model has a 10% chance to mutate each weight
        """
        self.input_size = 59
        self.output_size = 3
        self.mutation_rate = 0.1  # 10% chance to mutate

        if weights is None:
            self.weights = np.random.uniform(-1, 1, (self.output_size, self.input_size))
        else:
            self.weights = weights

    def sigmoid(self, x):
        return 1 / (1 + math.exp(-x))

    def feed_forward(self, inputs):
        return [self.sigmoid(i) for i in (inputs * self.weights).sum(axis=1)]

    def mutate(self):
        """When mutating, there is a 10% chance per weight to reinitialize (randomize) its value.
        """
        self.weights = pd.DataFrame(self.weights).apply(
            lambda ser: ser.apply(lambda x: np.random.uniform(-1, 1) if np.random.random() <= 0.1 else x)).values

    def save(self, file_name):
        np.save(file_name, self.weights)

### Controller

In [14]:
class Controller():
    def __init__(self):
        self.number_of_models = 5
        self.iteration_number = 1
        self.directory = "training/"
        self.times = []
        self.models = []
        if not os.path.exists(self.directory):
            os.makedirs(self.directory)
        else:
            self.load_previous_models()

    def load_previous_models(self):
        pass
            
    def initialize_training(self):
        self.training_set_dir = self.directory + str(self.iteration_number) + "/"
        os.mkdir(self.training_set_dir)
        
        for model_index in range(self.number_of_models):
            model = Model()
            model.save(self.training_set_dir + str(model_index) + ".npy")
            self.models.append(model)

    def train_step(self):
        self.scores = []
        """This is one step for training, each of the models runs the game, then total time is used as score
        
        Score is then scaled by dividing my total score value.
        """
        
        for model in self.models:
            start = time.time()
            end = time.time() - start
            self.scores.append(end)
        
        self.times.append(np.max(self.scores))
        
        self.scores = np.array(self.scores)
        self.scores = self.scores / np.sum(self.scores)
        
        self.iteration_number+=1
    
    def prepare_next_step(self):
        """Function called when models are finished training
        
        Randomly selects next models based on probability of total time
        Models are then mutated.
        """
        
        self.models = random.choices(self.models, self.scores, k=self.number_of_models)
        self.models = [model.mutate() for model in self.models]

In [15]:
controller = Controller()

In [16]:
controller.initialize_training()

In [48]:
a = [10,20,100,30,5]

In [49]:
a = np.array(a)

In [50]:
b = a / np.sum(a)

In [65]:
random.choices(a, b, k=10)

[100, 30, 30, 30, 30, 100, 100, 100, 100, 100]

In [9]:
controller.train_step()

In [21]:
import json

In [22]:
a = {1:2,3:4}

In [48]:
open("scores.json", "w+")

<_io.TextIOWrapper name='scores.json' mode='w+' encoding='cp1252'>

In [49]:
with open("scores.json") as f:
    d = json.load(f)
f.close()

with open("scores.json", "w+") as f:
    json.dump(d, f)
f.close()

JSONDecodeError: Expecting value: line 1 column 1 (char 0)